# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 14 2022 8:30AM,246347,10,0085978252,ISDIN Corporation,Released
1,Sep 14 2022 8:30AM,246347,10,0085978274,ISDIN Corporation,Released
2,Sep 14 2022 8:30AM,246347,10,0085978278,ISDIN Corporation,Released
3,Sep 14 2022 8:30AM,246347,10,0085978289,ISDIN Corporation,Released
4,Sep 14 2022 8:30AM,246347,10,0085978293,ISDIN Corporation,Released
5,Sep 14 2022 8:30AM,246347,10,0085978299,ISDIN Corporation,Released
6,Sep 14 2022 8:30AM,246347,10,0085978335,ISDIN Corporation,Released
7,Sep 14 2022 8:30AM,246347,10,0085978341,ISDIN Corporation,Released
8,Sep 14 2022 8:30AM,246347,10,0085978343,ISDIN Corporation,Released
9,Sep 14 2022 8:30AM,246347,10,0085978392,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,246366,Released,11
43,246367,Executing,1
44,246368,Released,4
45,246369,Released,1
46,246370,Released,58


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246366,NaN,NaN,11.0
246367,NaN,1.0,NaN
246368,NaN,NaN,4.0
246369,NaN,NaN,1.0
246370,NaN,NaN,58.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246178,0.0,1.0,0.0
246182,0.0,0.0,1.0
246185,0.0,0.0,1.0
246195,0.0,0.0,1.0
246199,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246178,0,1,0
246182,0,0,1
246185,0,0,1
246195,0,0,1
246199,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246178,0,1,0
1,246182,0,0,1
2,246185,0,0,1
3,246195,0,0,1
4,246199,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246178,,1,
1,246182,,,1
2,246185,,,1
3,246195,,,1
4,246199,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation
11,Sep 14 2022 8:30AM,246353,10,"Nextsource Biotechnology, LLC"
13,Sep 14 2022 8:30AM,246358,10,TherapeuticsMD Retain
14,Sep 14 2022 8:29AM,246370,10,ISDIN Corporation
72,Sep 14 2022 8:21AM,246369,19,ACG North America LLC
73,Sep 14 2022 8:18AM,246368,10,Emerginnova
77,Sep 14 2022 8:15AM,246367,22,"NBTY Global, Inc."
78,Sep 14 2022 8:15AM,246366,10,ISDIN Corporation
89,Sep 14 2022 7:47AM,246364,19,"AdvaGen Pharma, Ltd"
91,Sep 13 2022 7:24PM,246363,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,,,11
1,Sep 14 2022 8:30AM,246353,10,"Nextsource Biotechnology, LLC",,,2
2,Sep 14 2022 8:30AM,246358,10,TherapeuticsMD Retain,,,1
3,Sep 14 2022 8:29AM,246370,10,ISDIN Corporation,,,58
4,Sep 14 2022 8:21AM,246369,19,ACG North America LLC,,,1
5,Sep 14 2022 8:18AM,246368,10,Emerginnova,,,4
6,Sep 14 2022 8:15AM,246367,22,"NBTY Global, Inc.",,1,
7,Sep 14 2022 8:15AM,246366,10,ISDIN Corporation,,,11
8,Sep 14 2022 7:47AM,246364,19,"AdvaGen Pharma, Ltd",,,2
9,Sep 13 2022 7:24PM,246363,16,Sartorius Lab Products and Service,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11,,
1,Sep 14 2022 8:30AM,246353,10,"Nextsource Biotechnology, LLC",2,,
2,Sep 14 2022 8:30AM,246358,10,TherapeuticsMD Retain,1,,
3,Sep 14 2022 8:29AM,246370,10,ISDIN Corporation,58,,
4,Sep 14 2022 8:21AM,246369,19,ACG North America LLC,1,,
5,Sep 14 2022 8:18AM,246368,10,Emerginnova,4,,
6,Sep 14 2022 8:15AM,246367,22,"NBTY Global, Inc.",,1,
7,Sep 14 2022 8:15AM,246366,10,ISDIN Corporation,11,,
8,Sep 14 2022 7:47AM,246364,19,"AdvaGen Pharma, Ltd",2,,
9,Sep 13 2022 7:24PM,246363,16,Sartorius Lab Products and Service,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11,,
1,Sep 14 2022 8:30AM,246353,10,"Nextsource Biotechnology, LLC",2,,
2,Sep 14 2022 8:30AM,246358,10,TherapeuticsMD Retain,1,,
3,Sep 14 2022 8:29AM,246370,10,ISDIN Corporation,58,,
4,Sep 14 2022 8:21AM,246369,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11.0,NaN,NaN
1,Sep 14 2022 8:30AM,246353,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
2,Sep 14 2022 8:30AM,246358,10,TherapeuticsMD Retain,1.0,NaN,NaN
3,Sep 14 2022 8:29AM,246370,10,ISDIN Corporation,58.0,NaN,NaN
4,Sep 14 2022 8:21AM,246369,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11.0,0.0,0.0
1,Sep 14 2022 8:30AM,246353,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
2,Sep 14 2022 8:30AM,246358,10,TherapeuticsMD Retain,1.0,0.0,0.0
3,Sep 14 2022 8:29AM,246370,10,ISDIN Corporation,58.0,0.0,0.0
4,Sep 14 2022 8:21AM,246369,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2955973,94.0,0.0,0.0
102,985422,4.0,0.0,0.0
15,246227,29.0,0.0,90.0
16,492697,6.0,2.0,0.0
17,492723,2.0,0.0,0.0
19,1231637,14.0,8.0,21.0
20,738965,3.0,0.0,0.0
21,3200771,12.0,1.0,0.0
22,246367,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2955973,94.0,0.0,0.0
1,102,985422,4.0,0.0,0.0
2,15,246227,29.0,0.0,90.0
3,16,492697,6.0,2.0,0.0
4,17,492723,2.0,0.0,0.0
5,19,1231637,14.0,8.0,21.0
6,20,738965,3.0,0.0,0.0
7,21,3200771,12.0,1.0,0.0
8,22,246367,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,94.0,0.0,0.0
1,102,4.0,0.0,0.0
2,15,29.0,0.0,90.0
3,16,6.0,2.0,0.0
4,17,2.0,0.0,0.0
5,19,14.0,8.0,21.0
6,20,3.0,0.0,0.0
7,21,12.0,1.0,0.0
8,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,94.0
1,102,Released,4.0
2,15,Released,29.0
3,16,Released,6.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,17,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,90.0,0.0,0.0,21.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,2.0,0.0,8.0,0.0,1.0,1.0
Released,94.0,4.0,29.0,6.0,2.0,14.0,3.0,12.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,17,19,20,21,22
0,Completed,0.0,0.0,90.0,0.0,0.0,21.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,2.0,0.0,8.0,0.0,1.0,1.0
2,Released,94.0,4.0,29.0,6.0,2.0,14.0,3.0,12.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,17,19,20,21,22
0,Completed,0.0,0.0,90.0,0.0,0.0,21.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,2.0,0.0,8.0,0.0,1.0,1.0
2,Released,94.0,4.0,29.0,6.0,2.0,14.0,3.0,12.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()